In [8]:
HTML(readall(open("style.css")))

HTML{ASCIIString}("<style>\n\n/*div#maintoolbar, div#header {display: none !important;}\n*/\n#notebook-container {\nfont-family: Computer Modern; \n  width:700px;\n  padding: 20px;\n  background-color: #FFF;\n  min-height: 10;\n  -webkit-box-shadow: 0px 0px 0px 2px rgba(0, 0, 0, 1);\n  /* box-shadow: 0px 0px 2px 1px rgba(0, 0, 0, 1); */\n}\n\nbody {\n  background-color: white !important;\n}\n\n@font-face { \n  font-family: Computer Modern; \nsrc:url('http://gabgoh.github.io/cmunorm.ttf');\n}\n\n/* src: url('http://mirrors.ctan.org/\nfonts/cm-unicode/fonts/otf/cmunss.otf'); */\n\ndiv.cell{\n  width:600px;\n  margin-left:auto;\n  margin-right:auto;\n}\n\ndiv.output_subarea {\n    overflow-x: auto;\n    padding: 0.4em;\n    -webkit-box-flex: 1;\n    -moz-box-flex: 1;\n    box-flex: 1;\n    flex: 1;\n    max-width: calc(100%);\n}\n\nh1 { font-family: Computer Modern; }\n\n.prompt{ display:None; }\n\n.output_png { margin: auto; }\n\nhr {\n    display: block;\n    height: 1px;\n    border: 0;\n    border-top: 1px solid #000;\n    margin: 1em 0;\n    padding: 0; \n}\n\n</style>\n")

# KKT Callbacks with IntPoint

The `intpoint` solver can be accelerated when there is structure to be exploited in problem. Here we will consider the simple problem:

$$\mbox{minimize}\quad\frac{1}{2}x^{T}Qx-b^{T}x,\qquad\mbox{s.t.}\quad x\geq0.$$

The solver can be called with no special parameters,

In [2]:
using IntPoint

n = 1000

Q = sparse(randn(n,n)); 
Q = Q'*Q;
c = ones(n,1);
A = speye(n);
b = zeros(n,1);
𝐾 = [("R",n)];

@time intpoint( Q , c , A , b , 𝐾 , verbose = true);


 > INTERIOR POINT SOLVER v0.7 (July 2016)

   Iter  Mu          prFeas      duFeas      muFeas      refine    
      1  1.7952e+00  1.3283e+00  4.1735e+01  1.2551e+00  1
      2  4.0811e-01  2.8448e-01  8.9385e+00  3.4530e-01  1
      3  1.5700e-01  8.0023e-02  2.5144e+00  2.9434e-01  1
      4  4.5733e-02  1.8694e-02  5.8739e-01  1.3231e-01  1
      5  1.3577e-02  3.6356e-03  1.1423e-01  6.4463e-02  1
      6  1.9794e-03  1.5858e-14  5.9771e-17  1.0041e-02  1
      7  2.8059e-04  1.1036e-14  4.8422e-17  1.3830e-03  1
      8  2.7077e-05  1.0287e-14  4.7177e-17  1.4751e-04  1
      9  2.5619e-06  1.0359e-14  4.9834e-17  1.6370e-05  1
     10  1.3271e-07  1.0182e-14  4.5557e-17  1.4900e-06  1

 > EXIT -- Below Tolerance!

 13.895687 seconds (11.95 M allocations: 2.599 GB, 4.59% gc time)


The speed of the solver is reasonable, as the deault solver exploits the sparsity of the constraint matrix. We can do better, however.

___

## KKT Callbacks
The KKT callback requires a function `solve3x3gen` which returns a function solving the KKT system

$$\left(\begin{array}{ccc}
Q & G^{T} & A^{T}\\
G\\
A &  & -F^{2}
\end{array}\right)
\left(\begin{array}{c}
a\\
c\\
b
\end{array}\right) = \left(\begin{array}{c}
x\\
z\\
y
\end{array}\right)$$

In this example, since we have no linear constraints, $G$ is empty, and our KKT system is

$$
\left(\begin{array}{cc}
Q & I\\
I & -F^{-2}
\end{array}\right)\left(\begin{array}{c}
a\\
b
\end{array}\right)=\left(\begin{array}{c}
x\\
y
\end{array}\right)
$$

The system can be solved by pivoting on the second block, as follows:

$$
(Q+F^{-2})\,a=x+F^{-2}y,\qquad b=F^{-2}(a-y)
$$

Because we only have polyhedral constraints, $F^{-2}$ is just a diagonal matrix, thus the first equation is a digonal perturbation to $Q$ which can be solved via a Cholesky Factorization.

In [3]:
function solve3x3gen(F, F⁻¹, Q, A, G)

  F⁻² = F⁻¹^2
  QpD⁻¹ = cholfact(Q + spdiagm( (F[1].diag).^(-2) ))

  function solve3x3(x, z, y)

    a = QpD⁻¹\(x + A'*(F⁻²*y))
    b = F⁻²*(y - A*a)
    c = zeros(0,1)              # empty vector.
    return(a, c, b)

  end

end

@time sol = intpoint( Q , c , A , b , 𝐾 , 
    solve3x3gen = solve3x3gen; 
    verbose = true);


 > INTERIOR POINT SOLVER v0.7 (July 2016)

   Iter  Mu          prFeas      duFeas      muFeas      refine    
      1  1.7952e+00  1.3283e+00  4.1735e+01  1.2551e+00  1
      2  4.0811e-01  2.8448e-01  8.9385e+00  3.4530e-01  1
      3  1.5700e-01  8.0023e-02  2.5144e+00  2.9434e-01  1
      4  4.5733e-02  1.8694e-02  5.8739e-01  1.3231e-01  1
      5  1.3577e-02  3.6356e-03  1.1423e-01  6.4463e-02  1
      6  1.9794e-03  1.5195e-14  7.7694e-17  1.0041e-02  1
      7  2.8059e-04  1.0765e-14  5.6920e-17  1.3830e-03  1
      8  2.7077e-05  1.0680e-14  5.7174e-17  1.4751e-04  1
      9  2.5619e-06  1.0238e-14  5.3166e-17  1.6370e-05  1
     10  1.3271e-07  1.0034e-14  5.9159e-17  1.4900e-06  1

 > EXIT -- Below Tolerance!

  3.674023 seconds (1.25 M allocations: 1.088 GB, 8.23% gc time)


This results in a 5-fold improvement in speed, and a dramatic drop in memory usage!

This pattern of pivoting on the third block happens often enough that I have encapsulated it in the convenience function `pivot`, which transforms a $2x2$ solver of the system

$$
\left(\begin{array}{cc}
Q+A^{T}F^{-2}F & G\\
G & 0
\end{array}\right)\left(\begin{array}{c}
a\\
b
\end{array}\right)\mapsto\left(\begin{array}{c}
x\\
y
\end{array}\right)
$$

into a $3x3$ solver. This is illustrated below

In [4]:
function solve2x2gen(F, F⁻¹, Q, A, G)

  QpD⁻¹ = cholfact(Q + spdiagm( (F[1].diag).^(-2) ))
  return (y, x) -> (QpD⁻¹\y, zeros(0,1))

end

@time sol = intpoint( Q , c , A , b , 𝐾 , 
    solve3x3gen = pivot(solve2x2gen); 
    verbose = true);


 > INTERIOR POINT SOLVER v0.7 (July 2016)

   Iter  Mu          prFeas      duFeas      muFeas      refine    
      1  1.7952e+00  1.3283e+00  4.1735e+01  1.2551e+00  1
      2  4.0811e-01  2.8448e-01  8.9385e+00  3.4530e-01  1
      3  1.5700e-01  8.0023e-02  2.5144e+00  2.9434e-01  1
      4  4.5733e-02  1.8694e-02  5.8739e-01  1.3231e-01  1
      5  1.3577e-02  3.6356e-03  1.1423e-01  6.4463e-02  1
      6  1.9794e-03  1.5426e-14  7.8324e-17  1.0041e-02  1
      7  2.8059e-04  1.0992e-14  5.9428e-17  1.3830e-03  1
      8  2.7077e-05  1.0515e-14  5.4830e-17  1.4751e-04  1
      9  2.5619e-06  1.0239e-14  5.5022e-17  1.6370e-05  1
     10  1.3271e-07  1.0436e-14  5.5529e-17  1.4900e-06  1

 > EXIT -- Below Tolerance!

  1.577840 seconds (80.08 k allocations: 1.037 GB, 13.25% gc time)


And as a bonus we even get an extra boost in speed! 
___